In [1]:
# # header
import sys
sys.path.append("../")

%load_ext autoreload
%autoreload 2

In [2]:
# # imports
import torch
import os
import argparse
from pprint import pformat
from collections import OrderedDict
from src.composer import COCOTokens, COCOAnimals, BG20k
from src.composer import PerceptualGrouping_COCO, Recognition_COCO, Search_COCO, SearchGrid_COCO
from src.conductor import AttentionTrain
from src.modelv2 import AttentionModel
from src.utils import plot_all, plot_loss_all
from src.utils import build_loaders, get_n_parameters
from prelude import get_device, startup_folders, load_dicts


In [ ]:
start_folder = r"../pretrained/coco_v2"
results_folder, logger = startup_folders(start_folder, name=f"exp_coco")
data_path = r"../data"

../pretrained/coco_v2/1769695544 was created!


In [4]:
model_params = load_dicts(start_folder, "model_params")
tasks = load_dicts(start_folder, "tasks")
train_params = load_dicts(start_folder, "train_params")
DeVice, num_workers, pin_memory = get_device()
print(f"model_params: {model_params}")
print(f"tasks: {tasks}")
print(f"train_params: {train_params}")

Device set to mps
model_params: {'channels': [32, 64, 128, 256, 512], 'fun': GELU(approximate='none'), 'in_dims': [3, 256, 256], 'mid_dim': -1, 'n_classes': 10, 'n_tasks': 3, 'norm': 'layer', 'out_dim': 10, 'softness': 0.5, 'task_fun': Tanh(), 'bnpti': False, 'recurrent': False, 'concat': True, 'resnet': False}
tasks: {'Recognition': {'composer': 'Recognition_COCO', 'key': 0, 'params': {'n_iter': 3, 'stride': 64, 'blank': False, 'static': False, 'noise': 0.25}, 'datasets': ['Recognition_COCO', 'Recognition_COCO', 'Recognition_COCO'], 'dataloaders': [None, None, None], 'loss_w': [0.25, 0.0, 0.5], 'loss_s': '(slice(1, None, None), None)', 'class_weights': 'tensor([0.1306, 0.0649, 0.0684, 0.0739, 0.0956, 0.0905, 0.0757, 0.2579, 0.0753,\n        0.0672])', 'has_prompt': False}, 'PerceptualGrouping': {'composer': 'PerceptualGrouping_COCO', 'key': 1, 'params': {'fix_attend': [2, 2], 'noise': 0.25, 'natural': False}, 'datasets': ['PerceptualGrouping_COCO', 'PerceptualGrouping_COCO', 'Perceptu

In [5]:
tasks['Recognition']["composer"] = Recognition_COCO
tasks['PerceptualGrouping']["composer"] = PerceptualGrouping_COCO
tasks['Search']["composer"] = Search_COCO
tasks['SearchGrid']["composer"] = SearchGrid_COCO

tasks['Recognition']["datasets"] = []
tasks['PerceptualGrouping']["datasets"] = []
tasks['Search']["datasets"] = []
tasks['SearchGrid']["datasets"] = []

tasks['Recognition']["dataloaders"] = []
tasks['PerceptualGrouping']["dataloaders"] = []
tasks['Search']["dataloaders"] = []
tasks['SearchGrid']["dataloaders"] = []

tasks['Recognition']["loss_s"] = (slice(0, None, None), slice(0, None, None))
tasks['PerceptualGrouping']["loss_s"] = (slice(1, None, None), slice(1, None, None))
tasks['Search']["loss_s"] = (slice(1, None, None), slice(1, None, None))
tasks['SearchGrid']["loss_s"] = (slice(1, None, None), slice(1, None, None))

tasks['Recognition']["loss_s"] = ((-1, ), (-1, ))
tasks['PerceptualGrouping']["loss_s"] = ((-1, ), (-1, ))
tasks['Search']["loss_s"] = ((-1, ), (-1, ))
tasks['SearchGrid']["loss_s"] = ((-1, ), (-1, ))

In [6]:
# datasets and dataloaders
coco_tokens = COCOTokens(directory=data_path, animals=True, split=0.9)
train_tks, valid_tks, test_tks = coco_tokens.get_tokens()
train_coco = COCOAnimals(in_dims=model_params["in_dims"], directory=data_path, kind=0, tokens=train_tks)
valid_coco = COCOAnimals(in_dims=model_params["in_dims"], directory=data_path, kind=1, tokens=valid_tks)
test_coco = COCOAnimals(in_dims=model_params["in_dims"], directory=data_path, kind=2, tokens=test_tks)
train_bg = BG20k(root=data_path, kind="train")
test_bg = valid_bg = BG20k(root=data_path, kind="test")
DeVice, num_workers, pin_memory = get_device()
for o in tasks:
    if tasks[o]["composer"] in (Recognition_COCO , SearchGrid_COCO):
        tasks[o]["datasets"].append(tasks[o]["composer"](train_coco, **tasks[o]["params"], bg_dataset=train_bg))
        tasks[o]["datasets"].append(tasks[o]["composer"](valid_coco, **tasks[o]["params"], bg_dataset=valid_bg))
        tasks[o]["datasets"].append(tasks[o]["composer"](test_coco, **tasks[o]["params"], bg_dataset=test_bg))
    else:
        tasks[o]["datasets"].append(tasks[o]["composer"](train_coco, **tasks[o]["params"]))
        tasks[o]["datasets"].append(tasks[o]["composer"](valid_coco, **tasks[o]["params"]))
        tasks[o]["datasets"].append(tasks[o]["composer"](test_coco, **tasks[o]["params"]))
    tasks[o]["datasets"][1].build_valid_test()
    tasks[o]["datasets"][2].build_valid_test()
    tasks[o]["dataloaders"] = build_loaders(tasks[o]["datasets"], batch_size=train_params["batch_size"], num_workers=num_workers, pin_memory=pin_memory, shuffle=False)
assert model_params["n_classes"] == train_coco.n_classes, f"Number of n_classes {model_params['n_classes']} and n_classes {train_coco.n_classes} must be equal!"
tasks["PerceptualGrouping"]["class_weights"] = train_coco.class_weights if hasattr(train_coco, "class_weights") else None
tasks["Recognition"]["class_weights"] = train_coco.class_weights if hasattr(train_coco, "class_weights") else None


loading annotations into memory...
Done (t=4.95s)
creating index...
index created!
loading annotations into memory...
Done (t=0.22s)
creating index...
index created!
loading annotations into memory...
Done (t=4.89s)
creating index...
index created!
loading annotations into memory...
Done (t=5.22s)
creating index...
index created!
loading annotations into memory...
Done (t=0.24s)
creating index...
index created!
Device set to mps


In [7]:
# model and optimizer...
model = AttentionModel(**model_params)
model_trainer = AttentionTrain(model, None, None, tasks, logger, results_folder)
model_dir = os.path.join(start_folder, "model" + ".pth")
assert os.path.exists(model_dir), "Could not find the model.pth in the given dir!"
model.load_state_dict(torch.load(model_dir, map_location=DeVice))

<All keys matched successfully>

In [8]:
# plotting
plot_all(10, model, tasks, results_folder, "_test", DeVice, logger, False, "test")


In [9]:
# evaluating...
model_trainer.eval(DeVice, "test", False)
model_trainer.eval(DeVice, "valid", False)

testing...
  Task Recognition:
    CEi Loss: 0.044    CEe Loss: 0.044    Pix Err: 0.027    Att Acc: 0.958    Cls Acc: 1332/1558
  Task PerceptualGrouping:
    CEi Loss: 0.045    CEe Loss: 0.044    Pix Err: 0.020    Att Acc: 0.907    Cls Acc: 1325/1558
  Task Search:
    CEi Loss: 0.469    CEe Loss: 0.469    Pix Err: 0.024    Att Acc: 0.913    Cls Acc: 543/646
  Task SearchGrid:
    CEi Loss: 0.756    CEe Loss: 0.751    Pix Err: 0.011    Att Acc: 0.943    Cls Acc: 1183/1558
validating...
  Task Recognition:
    CEi Loss: 0.019    CEe Loss: 0.019    Pix Err: 0.026    Att Acc: 0.960    Cls Acc: 3112/3375
  Task PerceptualGrouping:
    CEi Loss: 0.024    CEe Loss: 0.022    Pix Err: 0.015    Att Acc: 0.930    Cls Acc: 3054/3375
  Task Search:
    CEi Loss: 0.284    CEe Loss: 0.278    Pix Err: 0.014    Att Acc: 0.948    Cls Acc: 1397/1545
  Task SearchGrid:
    CEi Loss: 0.519    CEe Loss: 0.507    Pix Err: 0.007    Att Acc: 0.962    Cls Acc: 2840/3375
